In [48]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
import shutil
import random
from shutil import copyfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import scipy
import dlib

In [2]:
source_path = 'data'

source_path_level_0 = os.path.join(source_path, 'Level_0')
source_path_level_1 = os.path.join(source_path, 'Level_1')
source_path_level_2 = os.path.join(source_path, 'Level_2')

print(f"There are {len(os.listdir(source_path_level_0))} images in level 0.")
print(f"There are {len(os.listdir(source_path_level_1))} images of level 1.")
print(f"There are {len(os.listdir(source_path_level_2))} images of level 2.")

There are 387 images in level 0.
There are 473 images of level 1.
There are 139 images of level 2.


In [3]:

root_dir = 'data/split_data'

if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# GRADED FUNCTION: create_train_val_dirs
def create_train_val_dirs(root_path):


  os.makedirs(root_path, exist_ok=True)

  train_dir = os.path.join(root_path, 'training')
  validation_dir = os.path.join(root_path, 'validation')

  train_level_0_dir = os.path.join(train_dir, 'Level_0')
  train_level_1_dir = os.path.join(train_dir, 'Level_1')
  train_level_2_dir = os.path.join(train_dir, 'Level_2')

  validation_level_0_dir = os.path.join(validation_dir, 'Level_0')
  validation_level_1_dir = os.path.join(validation_dir, 'Level_1')
  validation_level_2_dir = os.path.join(validation_dir, 'Level_2')

  os.makedirs(train_level_0_dir, exist_ok=True)
  os.makedirs(train_level_1_dir, exist_ok=True)
  os.makedirs(train_level_2_dir, exist_ok=True)
  os.makedirs(validation_level_0_dir, exist_ok=True)
  os.makedirs(validation_level_1_dir, exist_ok=True)
  os.makedirs(validation_level_2_dir, exist_ok=True)


try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [4]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

data/split_data/training
data/split_data/validation
data/split_data/training/Level_2
data/split_data/training/Level_0
data/split_data/training/Level_1
data/split_data/validation/Level_2
data/split_data/validation/Level_0
data/split_data/validation/Level_1


In [5]:
# grader-required-cell

# GRADED FUNCTION: split_data
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  ### START CODE HERE
  files = []
  for filename in os.listdir(SOURCE_DIR):
      file = SOURCE_DIR + filename
      if os.path.getsize(file) > 0:
          files.append(filename)
      else:
          print(f"{filename} is zero length, so ignoring.")


  random.shuffle(files)

  split_idx = int(SPLIT_SIZE * len(files))

  train_files = files[:split_idx]
  validation_files = files[split_idx:]

  for filename in train_files:
      source_file = os.path.join(SOURCE_DIR, filename)
      destination_file = os.path.join(TRAINING_DIR, filename)
      copyfile(source_file, destination_file)

  for filename in validation_files:
      source_file = os.path.join(SOURCE_DIR, filename)
      destination_file = os.path.join(VALIDATION_DIR, filename)
      copyfile(source_file, destination_file)


In [6]:
# grader-required-cell

# Test your split_data function

# Define paths

LEVEL_0_SOURCE_DIR = 'data/Level_0/'
LEVEL_1_SOURCE_DIR = 'data/Level_1/'
LEVEL_2_SOURCE_DIR = 'data/Level_2/'

TRAINING_DIR = "data/split_data/training/"
VALIDATION_DIR = "data/split_data/validation/"

TRAINING_LEVEL_0_DIR = os.path.join(TRAINING_DIR, "Level_0/")
VALIDATION_LEVEL_0_DIR = os.path.join(VALIDATION_DIR, "Level_0/")

TRAINING_LEVEL_1_DIR = os.path.join(TRAINING_DIR, "Level_1/")
VALIDATION_LEVEL_1_DIR = os.path.join(VALIDATION_DIR, "Level_1/")

TRAINING_LEVEL_2_DIR = os.path.join(TRAINING_DIR, "Level_2/")
VALIDATION_LEVEL_2_DIR = os.path.join(VALIDATION_DIR, "Level_2/")


# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_LEVEL_0_DIR)) > 0:
  for file in os.scandir(TRAINING_LEVEL_0_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_LEVEL_1_DIR)) > 0:
  for file in os.scandir(TRAINING_LEVEL_1_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_LEVEL_2_DIR)) > 0:
  for file in os.scandir(TRAINING_LEVEL_2_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_LEVEL_0_DIR)) > 0:
  for file in os.scandir(VALIDATION_LEVEL_0_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_LEVEL_1_DIR)) > 0:
  for file in os.scandir(VALIDATION_LEVEL_1_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_LEVEL_2_DIR)) > 0:
  for file in os.scandir(VALIDATION_LEVEL_2_DIR):
    os.remove(file.path)


split_size = .8

split_data(LEVEL_0_SOURCE_DIR, TRAINING_LEVEL_0_DIR, VALIDATION_LEVEL_0_DIR, split_size)
split_data(LEVEL_1_SOURCE_DIR, TRAINING_LEVEL_1_DIR, VALIDATION_LEVEL_1_DIR, split_size)
split_data(LEVEL_2_SOURCE_DIR, TRAINING_LEVEL_2_DIR, VALIDATION_LEVEL_2_DIR, split_size)

print(f"Training level 0's directory has {len(os.listdir(TRAINING_LEVEL_0_DIR))} images")
print(f"Training level 1's directory has {len(os.listdir(TRAINING_LEVEL_1_DIR))} images")
print(f"Training level 2's directory has {len(os.listdir(TRAINING_LEVEL_2_DIR))} images")

print(f"There are {len(os.listdir(TRAINING_LEVEL_0_DIR))} images of level 0 for training")
print(f"There are {len(os.listdir(TRAINING_LEVEL_1_DIR))} images of level 1 for training")
print(f"There are {len(os.listdir(TRAINING_LEVEL_2_DIR))} images of level 2 for training")
print(f"There are {len(os.listdir(VALIDATION_LEVEL_0_DIR))} images of level 0 for validation")
print(f"There are {len(os.listdir(VALIDATION_LEVEL_1_DIR))} images of level 1 for validation")
print(f"There are {len(os.listdir(VALIDATION_LEVEL_2_DIR))} images of level 2 for validation")

Training level 0's directory has 309 images
Training level 1's directory has 378 images
Training level 2's directory has 111 images
There are 309 images of level 0 for training
There are 378 images of level 1 for training
There are 111 images of level 2 for training
There are 78 images of level 0 for validation
There are 95 images of level 1 for validation
There are 28 images of level 2 for validation


In [7]:
def get_image_size(image_path):
  try:
    with Image.open(image_path) as image:
      width, height = image.size
      return width, height
  except IOError as e:
    print(f"Error opening image: {e}")
    return None 

In [8]:
image_path_1 = "data/Level_0/levle0_2.jpg"
image_path_2 = "data/Level_1/levle1_26.jpg"
image_path_3 = "data/Level_2/levle2_30.jpg"

width, height = get_image_size(image_path_1)
print(f"Image 1: width={width}, height={height}")

width, height = get_image_size(image_path_2)
print(f"Image 2: width={width}, height={height}")

width, height = get_image_size(image_path_3)
print(f"Image 3: width={width}, height={height}")

Image 1: width=500, height=555
Image 2: width=500, height=790
Image 3: width=500, height=592


In [77]:
def crop_image(image):
    # Perform cropping operation here
    cropped_image = image[50:150, 50:150, :]  # Example: crop a region of interest
    
    return cropped_image

In [95]:
def crop_image(image):
    # Implement your cropping logic here
    # For example, you can crop the center region of the image
    cropped_image = image[72:72+256, 72:72+256, :]  # Crop a region of 256x256 pixels from the center
    return cropped_image

def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    # Define the ImageDataGenerator for training data with cropping
    train_datagen = ImageDataGenerator(
        rescale=1/255,
        horizontal_flip=True,
        fill_mode='nearest',
        interpolation_order=3,
        zoom_range=0.2,
        preprocessing_function=crop_image
    )

    # Generate batches of training data
    train_generator = train_datagen.flow_from_directory(
        directory=TRAINING_DIR,
        batch_size=30,
        class_mode='sparse',
    )

    # Define the ImageDataGenerator for validation data without cropping
    validation_datagen = ImageDataGenerator(rescale=1/255)

    # Generate batches of validation data
    validation_generator = validation_datagen.flow_from_directory(
        directory=VALIDATION_DIR,
        batch_size=30,
        class_mode='sparse',
    )

    return train_generator, validation_generator

In [96]:
train_gen, val_gen = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 798 images belonging to 3 classes.
Found 201 images belonging to 3 classes.


In [97]:
def plot_images(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

sample_training_images, _ = next(train_gen)

plot_images(sample_training_images[:5])

ValueError: could not broadcast input array from shape (184,184,3) into shape (256,256,3)

In [86]:
# import efficientnet
from tensorflow.keras.applications import EfficientNetB0

# Create an instance of the model
model = EfficientNetB0( weights='imagenet',include_top=False, input_shape=(500, 500, 3))

# Freeze the pre-trained weights
model.trainable = False

# Add a global average pooling layer
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(model.input, x)



In [87]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])   
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 500, 500, 3)]        0         []                            
                                                                                                  
 rescaling_12 (Rescaling)    (None, 500, 500, 3)          0         ['input_8[0][0]']             
                                                                                                  
 normalization_7 (Normaliza  (None, 500, 500, 3)          7         ['rescaling_12[0][0]']        
 tion)                                                                                            
                                                                                                  
 rescaling_13 (Rescaling)    (None, 500, 500, 3)          0         ['normalization_7[0][0]'

In [88]:
history = model.fit(train_gen, validation_data=val_gen, epochs=10, verbose=1)

ValueError: could not broadcast input array from shape (100,100,3) into shape (500,500,3)

In [ ]:
#image, labels = next(train_gen)
#image.shape

In [ ]:
# def create_model():
#   model = tf.keras.models.Sequential([
#       tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(500, 500, 3)),
#       tf.keras.layers.MaxPooling2D(2,2),
#       tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2,2),
#       tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2,2),
#       tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2,2),
#       tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2,2),

#       tf.keras.layers.Flatten(),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(3, activation='softmax')
#   ])


#   model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy'])


#   return model

In [ ]:
# model = create_model()
# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 498, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 249, 249, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 247, 247, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 123, 123, 128)     0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 121, 121, 256)     295168    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 60, 60, 256)       0

In [ ]:
# history = model.fit(train_gen,
#                     epochs=15,
#                     verbose=1,
#                     validation_data=val_gen)

Epoch 1/15
11/40 [=======>......................] - ETA: 1:33 - loss: 1.4042 - accuracy: 0.3591

KeyboardInterrupt: 